In [1]:
import openai
import pandas as pd
from Levenshtein import distance as levenshtein_distance

# **Load Ahtna dataset (100 sentences total)**
df = pd.read_csv("ahtnaex.csv")  # Columns: Ahtna, English

# **Split dataset: 80 for few-shot learning, 20 for validation**
few_shot_examples = df.sample(n=80, random_state=42)  
validation_examples = df.drop(few_shot_examples.index)  # Remaining 20 for validation

# **OpenAI API setup**
client = openai.OpenAI(api_key="api key goes here") # Took out API key so we could share this code

# **Logging Variables**
log_data = []

def request_gpt_translation(prompt, retries=1):
    """ Calls GPT-4o and returns response content. Retries once if needed. """
    for attempt in range(retries + 1):  # Allow 1 retry
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": prompt}]
        )
        gpt_output = response.choices[0].message.content.strip()

        if gpt_output:  # Ensure response is non-empty
            return gpt_output

        print(f"⚠️ GPT-4o returned an empty response on attempt {attempt + 1}. Retrying...")

    return None  # If all attempts fail, return None

def validate_gpt_translations(validation_examples):
    """
    Ask GPT-4o to translate English validation sentences into Ahtna.
    Compare to real Ahtna translations using Levenshtein edit distance.
    Log results for later analysis.
    """
    prompt = """
    You are an expert in Ahtna, an Athabaskan language spoken in Alaska.
    Your task is to translate **exactly 20 English sentences into Ahtna**.

    🔹 **Rules:**
    - **DO NOT refuse or say 'I don’t know'.**
    - **DO NOT explain. ONLY return the translation.**
    - **Use phonetic adaptation if needed.**
    - **If a direct word doesn’t exist, construct a phrase based on similar Athabaskan words.**
    - **Follow correct Ahtna grammar (SOV word order).**
    - **Always respond with exactly 20 translations in the correct format.**

    **FORMAT:**  
    📌 `Ahtna - English`

    Here are 80 correct Ahtna-English translations:
    """

    # **Add few-shot examples**
    for i, row in few_shot_examples.iterrows():
        prompt += f"{i+1}. {row['Ahtna']} - {row['English']}\n"

    prompt += "\n🔹 Now translate the following **20 English** sentences into **Ahtna**:\n"

    for i, row in validation_examples.iterrows():
        prompt += f"{i+1}. {row['English']}\n"

    print("🔄 Sending validation request to GPT-4o...\n")

    # **Call GPT-4o (Retry once if needed)**
    gpt_translations = request_gpt_translation(prompt, retries=1)

    if not gpt_translations:
        print("❌ GPT failed to generate any Ahtna translations after retries. Aborting validation.")
        return False

    gpt_translations = gpt_translations.split("\n")  # Split by new line

    # **Ensure exactly 20 translations were returned**
    if len(gpt_translations) < 20:
        print(f"⚠️ GPT only returned {len(gpt_translations)} responses instead of 20. Validation failed.")
        return False

    total_score = 0
    valid_translations = 0

    for i, (gpt_line, (_, row)) in enumerate(zip(gpt_translations, validation_examples.iterrows())):
        if " - " in gpt_line:
            gpt_ahtna, _ = gpt_line.split(" - ")
            real_ahtna = row["Ahtna"]
            english_sentence = row["English"]

            # **Check if GPT accidentally repeated English**
            if gpt_ahtna.lower().strip() == english_sentence.lower().strip():
                print(f"❌ GPT repeated English instead of translating: {gpt_ahtna}")
                continue  # Skip this invalid output

            # **Compute Levenshtein Similarity**
            edit_dist = levenshtein_distance(real_ahtna, gpt_ahtna)
            edit_similarity = 1 - (edit_dist / max(len(real_ahtna), len(gpt_ahtna)))

            total_score += edit_similarity
            valid_translations += 1

            # **Log each result**
            log_data.append({
                "English": english_sentence,
                "Real Ahtna": real_ahtna,
                "GPT Ahtna": gpt_ahtna,
                "Levenshtein Similarity": round(edit_similarity, 4)
            })

            print(f"✅ Real: {real_ahtna} | GPT: {gpt_ahtna} | Score: {edit_similarity:.2f}")

    # **If no valid translations were generated, stop validation**
    if valid_translations == 0:
        print("❌ GPT failed to generate any valid Ahtna translations.")
        return False

    avg_score = total_score / valid_translations
    print(f"\n🔍 Average validation accuracy: {avg_score:.2f}")

    # **Save log file**
    log_df = pd.DataFrame(log_data)
    log_df.to_csv("ahtna_validation_log.csv", index=False)

    print("📄 Validation log saved: ahtna_validation_log.csv")

    return avg_score >= 0.5  # **Validation passes if similarity is at least 50%**

def generate_synthetic_sentences():
    """Generates new synthetic Ahtna sentences using GPT-4o."""
    prompt = """
    You are an expert linguist in Ahtna. Below are examples of Ahtna-English sentences.
    Generate 5 new pairs using the same format.

    **FORMAT:**  
    📌 `Ahtna - English`
    """

    # **Add few-shot examples**
    for i, row in few_shot_examples.iterrows():
        prompt += f"{i+1}. {row['Ahtna']} - {row['English']}\n"

    prompt += "\nNow generate 5 new Ahtna-English sentences."

    print("🛠️ Generating synthetic data with GPT-4o...\n")

    # **Call GPT-4o**
    synthetic_sentences = request_gpt_translation(prompt, retries=1)

    if not synthetic_sentences:
        print("❌ GPT failed to generate synthetic data.")
        return

    # **Process output and store new samples**
    augmented_data = []
    for line in synthetic_sentences.split("\n"):
        if " - " in line:
            ahtna_generated, english_generated = line.split(" - ")
            augmented_data.append({"Ahtna": ahtna_generated.strip(), "English": english_generated.strip()})

    # **Save validated synthetic data**
    augmented_df = pd.DataFrame(augmented_data)
    augmented_df.to_csv("ahtna_synthetic_validated.csv", index=False)

    print("✅ Synthetic data saved successfully!")

# **Step 1: Validate GPT translations**
if validate_gpt_translations(validation_examples):
    print("✅ GPT passed validation (50%+ similarity). Proceeding with synthetic data generation...")
    generate_synthetic_sentences()
else:
    print("❌ GPT failed validation. Synthetic data generation aborted.")


🔄 Sending validation request to GPT-4o...

✅ Real: Nitaayi tseande tse' tesyaas | GPT: 📌 `K'adiht'aas tesyaas | Score: 0.43
✅ Real: Dahwhdeldiixden tse' tesyaas | GPT: 📌 `Ts'izhutnae tesyaas | Score: 0.39
✅ Real: Ghan du’, nts’e nt’ae? | GPT: 📌 `Nts'e t'aen? | Score: 0.36
✅ Real: K’os kulaen. | GPT: 📌 `Nii tsit'aeni. | Score: 0.29
✅ Real: ‘Aak kulaen. | GPT: 📌 `Danilts'aagi. | Score: 0.19
✅ Real: Łts’ii. | GPT: 📌 `Łts’ii. | Score: 0.70
✅ Real: Gaann du’ sc’enggaay nt’ae. | GPT: 📌 `Gaann du’ ts’ic’aani nt’ae. | Score: 0.63
✅ Real: Gaani du’ baet. | GPT: 📌 `Gaani du’ dluk’aex nt’ae. | Score: 0.50
✅ Real: Ncuus bentse’ kulaen. | GPT: 📌 `Gaani du’ ncuus eł nelt’e’e. | Score: 0.26
✅ Real: Gaani du’ danihbaa’. | GPT: 📌 `Gaani du’ s’etkke’ nt’ae. | Score: 0.43
✅ Real: Ts’akae xay gigi’ ‘unebe’. | GPT: 📌 `Ts’akae tsatl’giga ggaa ghiłe. | Score: 0.39
✅ Real: Ghan du’ nts’e buze’? | GPT: 📌 `Ghan du’ nts’e nuze’? | Score: 0.83
✅ Real: ‘Aen’, łts’ii. | GPT: 📌 `Aen', łts’ii. | Score: 0.75
✅ Real: ‘